In [ ]:
#| default_exp process

# Process

> API details.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import mmcv
from loguru import logger
from IPython import display
def multi_thread(fn, array_inputs, max_workers=None, 
                 desc="", unit="Samples", 
                 verbose=True, pbar_iterval=10):
    from concurrent.futures import ThreadPoolExecutor
    from functools import partial
    from glob import glob
    from multiprocessing import Pool
    from tqdm import tqdm

    def _wraper(x):
        i, input = x
        return {i: fn(input)}

    array_inputs = [(i, _) for i, _ in enumerate(array_inputs)]
    if verbose:
        logger.info(desc)
        progress_bar = mmcv.ProgressBar(len(array_inputs))#tqdm(total=len(array_inputs))
    outputs = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i, result in enumerate(executor.map(_wraper, array_inputs)):
            outputs.update(result)
            if verbose and i%pbar_iterval==0:
                progress_bar.update(pbar_iterval)
    if verbose:
        logger.info('multi_thread {}, {}', fn.__name__, desc)
    outputs = list(outputs.values())
    return outputs




In [ ]:
#| hide
show_doc(multi_thread)

---

#### multi_thread

>      multi_thread (fn, array_inputs, max_workers=None, desc='',
>                    unit='Samples', verbose=True, pbar_iterval=10)

In [ ]:
#|export
def multi_process(f, inputs, max_workers=8, desc='',
               unit='Samples', verbose=True, pbar_iterval=10):
    from multiprocessing import Pool
    from tqdm import tqdm
    if verbose:
        pbar = mmcv.ProgressBar(len(inputs))
        logger.info('Multi processing {} | Num samples: {}', f.__name__, len(inputs))
        
    with Pool(max_workers) as p:
        it = p.imap(f, inputs)
        total = len(inputs)
        # return list(tqdm(it, total=total))
        return_list = []
        for i, ret in enumerate(it):
            return_list.append(ret)
            if i % pbar_iterval == 0:
                pbar.update(pbar_iterval)
    return return_list



---

#### multi_process

>      multi_process (f, inputs, max_workers=8, desc='', unit='Samples',
>                     verbose=True, pbar_iterval=10)

In [ ]:
#|hide
show_doc(multi_process)

---

#### multi_process

>      multi_process (f, inputs, max_workers=8, desc='', unit='Samples',
>                     verbose=True, pbar_iterval=10)

In [ ]:
#| hide
import numpy as np

import time
import mmcv
def f(x):
    time.sleep(np.random.uniform(0.1, .5))
multi_process(f, range(20), );

[                                                  ] 0/20, elapsed: 0s, ETA:

2022-09-23 06:20:30.262 | INFO     | __main__:multi_process:8 - Multi processing f | Num samples: 20


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 20/20, 36.8 task/s, elapsed: 1s, ETA:     0s

## Example

In [ ]:
show_doc(multi_thread)


def f(i):
    time.sleep(0.1)
    return i
timer = mmcv.Timer()

multi_thread(f, range(6), max_workers=2)
assert timer.since_start() < 2
print(f'Since start: {timer.since_start():0.2f} s')
multi_process(f, range(10))

2022-09-23 06:20:31.105 | INFO     | __main__:multi_thread:20 - 


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/6, 98.9 task/s, elapsed: 0s, ETA:     0s

2022-09-23 06:20:31.409 | INFO     | __main__:multi_thread:29 - multi_thread f, 


Since start: 0.31 s
[                                                  ] 0/10, elapsed: 0s, ETA:

2022-09-23 06:20:31.411 | INFO     | __main__:multi_process:8 - Multi processing f | Num samples: 10


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 63.3 task/s, elapsed: 0s, ETA:     0s

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
!nbdev_export